In [4]:
import json
import os
import pandas as pd
import re
from tqdm import tqdm

In [7]:
df =  pd.read_csv("partitionKarina.csv")

In [26]:
df

,id,chunks,chunks_embedded
0,<18709173.1075843349642.JavaMail.evans@thyme>,represents public power agencies around countr...,"[-0.00836747232824564, -0.03582817316055298, -..."
1,<6501090.1075843349674.JavaMail.evans@thyme>,FYI. Commentary calls George W. Bush task gett...,"[0.050551049411296844, -0.00414160406216979, 0..."
2,<6501090.1075843349674.JavaMail.evans@thyme>,"inauguration, President Bush's electricity pol...","[-0.02868984453380108, -0.0690743625164032, -0..."
3,<6501090.1075843349674.JavaMail.evans@thyme>,Energy Regulatory Commission (FERC) signals FE...,"[-0.006476814858615398, -0.03465692698955536, ..."
4,<6501090.1075843349674.JavaMail.evans@thyme>,California. Skyrocketing retail electric price...,"[-0.03591321036219597, -0.06088506430387497, -..."
...,...,...,...
165494,<24675364.1075840403182.JavaMail.evans@thyme>,5wEBgEpA5gEAAMA/5gEBAC1A5gEB0GNA5QEBADvA5gEAAN...,"[0.07229011505842209, -0.05499624088406563, 0...."
165495,<24675364.1075840403182.JavaMail.evans@thyme>,UAAjANcB+2qIKixFpD8DAg4AUAAkANcBwszUuPJKpT8DAg...,"[0.06907916069030762, -0.029502516612410545, 0..."
165496,<24675364.1075840403182.JavaMail.evans@thyme>,UAA8AOABAAAAAL4ANgBQAD0AkgGTAZIBkgGVAZUBlAGSAZ...,"[0.077231764793396, -0.023342903703451157, 0.0..."
165497,<24675364.1075840403182.JavaMail.evans@thyme>,MEADAg4AUQAaAM4B8B+P9tqk2D8DAg4AUQAbAPYBVIHutt...,"[0.055783797055482864, -0.029094845056533813, ..."


In [8]:
pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 57.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 93.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 67.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 80.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 90.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 79.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 91.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 89.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━

In [10]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from tqdm import tqdm

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Charger le modèle LaBSE et le tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')
model = AutoModel.from_pretrained('sentence-transformers/LaBSE').to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # Sorties des tokens
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def compute_embeddings_batch(batch_texts):
    # Encoder les textes avec le tokenizer
    encoded_input = tokenizer(batch_texts, padding=True, truncation=True, max_length=512, return_tensors='pt').to(model.device)
    with torch.no_grad():
        # Passer les entrées dans le modèle LaBSE
        model_output = model(**encoded_input)
    # Appliquer le mean pooling pour obtenir les embeddings des phrases
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return F.normalize(sentence_embeddings, p=2, dim=1).cpu().tolist()

def process_in_batches(df, batch_size=500):
    all_embeddings = []
    for i in tqdm(range(0, len(df), batch_size)):
        batch_texts = df['chunks'][i:i+batch_size].tolist()
        batch_embeddings = compute_embeddings_batch(batch_texts)
        all_embeddings.extend(batch_embeddings)
    return all_embeddings

In [12]:
# Calculer les embeddings par lots
df['chunks_embedded'] = process_in_batches(df, batch_size=200)

100%|██████████| 828/828 [59:05<00:00,  4.28s/it] 


In [13]:
df.to_csv('embedded_emails_LaBSE.csv', index=False)

In [14]:
# Télécharger le fichier dans Jupyter Notebook
from IPython.display import FileLink

# Générer un lien pour télécharger le fichier
display(FileLink('embedded_emails_LaBSE.csv'))

/home/onyxia/work/embedded_emails_LaBSE.csv

In [15]:
pip install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 48.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 18.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53771 sha256=fab196a430d924aa4f2737ebcba0e7573ff98c7fac9947774553590b62e6bca7
  Stored in directory: /home/onyxia/.cache/pip/wheels/d5/3d/69/8d68d249cd3de2584f226e27fd431d6344f7d70fd856ebd01b
Successfully built pypika
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib

In [16]:
import chromadb
from chromadb.config import Settings

In [17]:
# Charger le fichier contenant les embeddings et les chunks
df = pd.read_csv("embedded_emails_LaBSE.csv")

In [19]:
from chromadb.config import Settings
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import ast

In [21]:
# Initialiser ChromaDB
client = chromadb.Client()
collection = client.create_collection("mycollection")

In [22]:
batch_size = 500
for batch_start in range(0, len(df), batch_size):
    batch = df.iloc[batch_start:batch_start + batch_size]

    # Convertir les embeddings sous forme de chaînes en listes
    embeddings = batch['chunks_embedded'].apply(ast.literal_eval).tolist()
    documents = batch['chunks'].tolist()
    metadatas = [{'id': str(mail_id)} for mail_id in batch.get('id', range(len(batch))).tolist()]
    chunk_ids = [f"{batch_start + idx}" for idx in range(len(batch))]

    # Ajouter au collection ChromaDB
    collection.add(
        metadatas=metadatas,
        ids=chunk_ids,
        documents=documents,
        embeddings=embeddings
    )


In [23]:
print(f"Collection chargée avec {collection.count()} documents.")

Collection chargée avec 165499 documents.


In [24]:

# Charger le modèle LaBSE pour encoder une nouvelle phrase
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')
model = AutoModel.from_pretrained('sentence-transformers/LaBSE').to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def encode_sentence(sentence):
    encoded_input = tokenizer([sentence], padding=True, truncation=True, max_length=512, return_tensors='pt').to(model.device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    return F.normalize(sentence_embedding, p=2, dim=1).cpu().tolist()



In [25]:
# Exemple de phrase à tester
query_sentence = "What was the impact of President Bush's electricity policy on California's crisis?"
query_embedding = encode_sentence(query_sentence)

# Rechercher les 5 documents les plus proches dans la collection
results = collection.query(
    query_embeddings=query_embedding,
    n_results=5
)

# Afficher les résultats
print("Les 5 documents les plus proches de votre requête :")
for i in range(len(results['documents'][0])):
    print(f"\nDocument {i+1}:")
    print(f"ID: {results['ids'][0][i]}")
    print(f"Texte: {results['documents'][0][i]}")
    print(f"Métadonnées: {results['metadatas'][0][i]}")

Les 5 documents les plus proches de votre requête :

Document 1:
ID: 75055
Texte: benefited Bush's first 100 days: 2 - BUSH TAKES HANDS-OFF APPROACH CALIFORNIA CRISIS ENERGY COMPANIES MAKE MILLIONS Bush Little Aid California Energy Crisis; Fleischer Said Crisis "California Matter." Bush done little aid California energy crisis, refusing support wholesale price caps electricity. White House spokesman Ari Fleischer said, "The president continues believe issue mostly California matter, dealing legislation state. leaders California working address right." Fleischer also said Bush
Métadonnées: {'id': '<961551.1075843676155.JavaMail.evans@thyme>'}

Document 2:
ID: 111143
Texte: power generation facilities California. "I extremely gratified President Bush acted put federal government path California cut red tape get new power plants line," said Governor. ###
Métadonnées: {'id': '<21649802.1075843853086.JavaMail.evans@thyme>'}

Document 3:
ID: 62136
Texte: President Bush Gov. Gray Davis, sparr